#Apache Spark Class: Part 1 - Introduction
In the last section, we covered the steps needed to install Spark, if you haven't done that then do that now before moving forward.

In this section we cover the following:<br>
1. What is Apache Spark?
2. Importing the Spark Context
3. Brief intro to map and reduce with wordcounts
4. Explaination of transformation and action functions
5. Explaining lazy evaluation

##What is Apache Spark?
Apache Spark is a distributed computing framework that can handle many types of large-scale data processing tasks.  

In a very short time, Apache Spark has emerged as the next generation big data processing engine, and is being applied in production environemts as a replacement for many first generation "Big Data" tools. Spark improves over Hadoop MapReduce in several key dimensions:
* it is much faster due to its ability to store data in memory; some tasks can be sped up more than 100x 
* much easier to use due to its rich Java, Scala, and Python APIs 
* it goes far beyond batch applications to support a variety of workloads, including interactive queries, streaming, machine learning, and graph processing

Spark has a core processing engine, as well as four main libraries:<br>
<img src="https://spark.apache.org/images/spark-stack.png">

## Importing the Spark Context
The Spark Context is the root of the Spark Python API.  To access it in the notebook, we need to import it

In [1]:
from pyspark import SparkContext, SparkConf

###Configure Spark Settings
The SparkConf() class is the main class used for setting up our Spark app.  If you hit tab after conf., you will see a list of all settings available

In [2]:
conf=SparkConf()
conf.setAppName("My App")

###Initialize SparkContext
Once we have configurations set, we can create a spark context using the SparkContext class. Note that the 'local[*]' argument means that we are executing the code on our local machine and the * means use all available threads for computation (very important to include)

In [3]:
sc = SparkContext('local[*]', conf=conf)

You can now also go to http://localhost:4040/jobs/ and see the Spark App that you set up along with the progress of any jobs running

Printing the spark context shows that spark is connected to our notebook.  It should print something like:
```
<pyspark.context.SparkContext object at 0x7ffcaa3204d0>
```

In [29]:
print sc

In [ ]:
sc. #hitting tab when the cursor is after the period to see what functions are available with the context, try it

##Brief intro to map and reduce with wordcounts
Let's do a quick example to get used to using the sc.  In this case we want to get a count for how many times each word in the README file is used.

###Create an RDD
To make an RDD (Resilient Distributed Dataset, a spark data structure) called readme, read in a text file in the spark directory

In [30]:
#readme = sc.textFile('/home/<YOUR USERNAME>/spark/spark-1.3.0/README.md') #change <YOUR USERNAME> to your actual un
readme = sc.textFile('/home/scott/Documents/spark-1.3.0/README.md')

In [ ]:
readme. # hitting tab when the cursor is after the period will reveal all of the functions we can use on the RDD

Note that these mostly break down into two types: transformation and action functions, but we will dive into that later

###See what's there
Let's do a simple count the number of lines in the file:

In [6]:
readme.count()

98

Now let's take a look at what the first few lines look like in the RDD

In [31]:
readme.take(5)

[u'# Apache Spark',
 u'',
 u'Spark is a fast and general cluster computing system for Big Data. It provides',
 u'high-level APIs in Scala, Java, and Python, and an optimized engine that',
 u'supports general computation graphs for data analysis. It also supports a']

What we see is a "list-like" representation of the first five lines of the text file.  Note this not actually a list, it is an RDD so we will have to treat it like one using the functions available

###Counting the words
Now let's do our first manipulation of the RDD using map and a lambda function.  If you are not familiar with lambda functions, now would be a good time to read about them: https://pythonconquerstheuniverse.wordpress.com/2011/08/29/lambda_tutorial/ Essentially what we are doing here, is saying: for each line in the RDD, execute (the map part) the function split() which just splits the string at each space

In [10]:
word_lines = readme.map(lambda x: x.split())

Taking a look at the result, we can see it worked and we now have each row in the RDD split into individual words

In [13]:
word_lines.take(3)

[[u'#', u'Apache', u'Spark'],
 [],
 [u'Spark',
  u'is',
  u'a',
  u'fast',
  u'and',
  u'general',
  u'cluster',
  u'computing',
  u'system',
  u'for',
  u'Big',
  u'Data.',
  u'It',
  u'provides']]

Unfortunately there is one problem: we want all the words to be on their own individual row, and not one line per line in the file.  Fortunately, spark has something called flatMap, which is very similar to map, but does a final operation that takes the result of split() and puts each word from each line into its own row

In [17]:
words = readme.flatMap(lambda x: x.split())

In [20]:
words.take(10)

[u'#',
 u'Apache',
 u'Spark',
 u'Spark',
 u'is',
 u'a',
 u'fast',
 u'and',
 u'general',
 u'cluster']

Here we can see it worked; let's just do a quick double check by counting how many words there are

In [19]:
words.count()

458

Now the somewhat tricky part: we want to count how many times each word appears.  To do this we will first take each word, and put it in a tuple that looks like: (word, 1) where the 1 is just a static placeholder integer

In [21]:
wordcounts = words.map(lambda x: (x, 1))

In [22]:
wordcounts.take(5)

[(u'#', 1), (u'Apache', 1), (u'Spark', 1), (u'Spark', 1), (u'is', 1)]

Now for the secret sauce: reduceByKey.  We will now try and add up all the words to get a total count because we have things stored in a key, value format (word, 1), we can combine all the keys (aka words) that are the same while also adding thier values we do this using a lambda function with two arguments x and y which represent the key's current value plus the new value.  For example if we are mid way through adding the values and have (Apache, 3) and want to add one more (Apache, 1) we would end up with x = 3, y = 1 and the result is (Apache, 3+1) or (Apache, 4)

In [23]:
wordcounts = wordcounts.reduceByKey(lambda x,y:x+y)

In [24]:
wordcounts.take(5)

[(u'all', 1),
 (u'when', 1),
 (u'"local"', 1),
 (u'including', 3),
 (u'computation', 1)]

The result so far looks good, but we also want to see the words sorted by top counts to do this we will map a function that switches places for the word and count and then chain that with another function that sorts it in descending order

In [25]:
wordcounts = wordcounts.map(lambda x:(x[1],x[0])).sortByKey(False)

In [26]:
wordcounts.take(10)

[(21, u'the'),
 (14, u'Spark'),
 (14, u'to'),
 (11, u'for'),
 (10, u'and'),
 (9, u'a'),
 (8, u'##'),
 (7, u'run'),
 (6, u'is'),
 (6, u'on')]

###Chaining Transformations
The result looks pretty good! 'the' is the most frequent word one thing to note in that last map and sort is that we can chain transformations in one big list of things to do and get the same result

In [27]:
wordcounts = readme.flatMap(lambda x: x.split()).map(lambda x: (x, 1)).reduceByKey(lambda x,y:x+y).map(lambda x:(x[1],x[0])).sortByKey(False)

In [28]:
wordcounts.take(10)

[(21, u'the'),
 (14, u'Spark'),
 (14, u'to'),
 (11, u'for'),
 (10, u'and'),
 (9, u'a'),
 (8, u'##'),
 (7, u'run'),
 (6, u'is'),
 (6, u'on')]

Congrats, you did your first analysis using Spark!
<img src="https://s3.amazonaws.com/lowres.cartoonstock.com/education-teaching-word_count-essay-expression-idiom-phrase-kwan201_low.jpg">

##Explaination of transformation and action functions

### RDD Operations
As we’ve discussed, RDDs support two types of operations: transformations and actions. Transformations are operations on RDDs that return a new RDD, such as map() and filter(). Actions are operations that return a result to the driver program or write it to storage, and kick off a computation, such as count() and first(). Spark treats transformations and actions very differently, so understanding which type of operation you are performing will be important. If you are ever confused whether a given function is a transformation or an action, you can look at its return type: transformations return RDDs, whereas actions return some other data type.

###Transformations
Transformations are operations on RDDs that return a new RDD. Transformed RDDs are computed lazily, only when you use them in an action (more on that later). Many transformations are element-wise; that is, they work on one element at a time; but this is not true for all transformations.

Here are some examples of some of the most interesting or useful transformations:

In [5]:
example_rdd = sc.parallelize([1,1,2,3,5,8,11,19])

In [14]:
#map - Apply a function to each element in the RDD and return an RDD of the result.
map_example = example_rdd.map(lambda x: (x + 1, 'a'))
map_example.take(5)

[(2, 'a'), (2, 'a'), (3, 'a'), (4, 'a'), (6, 'a')]

In [18]:
#flatMap - Apply a function to each element in the RDD and return an RDD of the contents 
#of the iterators returned. Often used to extract words.
flatmap_example = map_example.flatMap(lambda x: str(x[0]))
flatmap_example.take(5)

['2', '2', '3', '4', '6']

In [19]:
#filter - Return an RDD consisting of only elements that pass the condition passed tofilter().
filter_example = example_rdd.filter(lambda x: x % 2 == 0)
filter_example.take(5)

[2, 8]

In [20]:
#distinct - Remove duplicates.
distinct_example = example_rdd.distinct()
distinct_example.collect()

[8, 1, 5, 2, 19, 11, 3]

In [24]:
#sample - Sample an RDD, with or without replacement.
sample_example = example_rdd.sample(False, 0.4)
sample_example.collect()

[1, 5, 11, 19]

In [25]:
#union - Produce an RDD containing elements from both RDDs.
union_example = example_rdd.union(distinct_example)
union_example.collect()

[1, 1, 2, 3, 5, 8, 11, 19, 8, 1, 5, 2, 19, 11, 3]

In [26]:
#intersection
intersection_example = filter_example.intersection(union_example)
intersection_example.collect()

[2, 8]

In [27]:
#subtract
subtract_example = union_example.subtract(intersection_example)
subtract_example.collect()

[1, 1, 1, 3, 3, 5, 5, 11, 11, 19, 19]

###Actions
We’ve seen how to create RDDs from each other with transformations, but at some point, we’ll want to actually do something with our dataset. Actions are the second type of RDD operation. They are the operations that return a final value to the driver program or write data to an external storage system. Actions force the evaluation of 
the transformations required for the RDD they were called on, since they need to actually produce output.

In [29]:
#take - Return all elements from the RDD.
example_rdd.take(3)

[1, 1, 2]

In [30]:
#count - Number of elements in the RDD.
example_rdd.count()

8

In [31]:
#countByValue - Number of times each element occurs in the RDD.
map_example.countByValue()

defaultdict(<type 'int'>, {(9, 'a'): 1, (12, 'a'): 1, (3, 'a'): 1, (6, 'a'): 1, (20, 'a'): 1, (2, 'a'): 2, (4, 'a'): 1})

In [32]:
#take - Return num elements from the RDD.
example_rdd.take(5)

[1, 1, 2, 3, 5]

In [33]:
#top - Return the top num elements the RDD.
example_rdd.top(6)

[19, 11, 8, 5, 3, 2]

In [36]:
#takeOrdered(num)(ordering) - Return num elements based on provided ordering.
example_rdd.takeOrdered(3, lambda x: -x )

[19, 11, 8]

In [38]:
#takeSample - Return num elements at random.
example_rdd.takeSample(False, 2)

[1, 8]

In [39]:
#reduce - Combine the elements of the RDD together in parallel (e.g.,sum).
example_rdd.reduce(lambda x, y: x + y)

50

In [40]:
#fold - Same as reduce() but with the provided zero value.
example_rdd.fold(0, lambda x, y: x + y)

50

In [41]:
#aggregate - Similar to reduce() but used to return a different type.
seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))
example_rdd.aggregate((0, 0), seqOp, combOp)

(50, 8)

In [43]:
#foreach - Apply the provided function to each element of the RDD.
def f(x): print x
example_rdd.foreach(f)

##Explaining lazy evaluation

###Lazy Evaluation
<p>Transformations on RDDs are lazily evaluated, meaning that Spark will not begin to execute until it sees an action. This can be somewhat counter intuitive for new users, but may be familiar for those who have used functional languages such as Haskell or LINQ-like data processing frameworks.</p>

<p>Lazy evaluation means that when we call a transformation on an RDD (for instance, calling map()), the operation is not immediately performed. Instead, Spark internally records metadata to indicate that this operation has been requested. Rather than thinking of an RDD as containing specific data, it is best to think of each RDD as consisting of instructions on how to compute the data that we build up through transformations. Loading data into an RDD is lazily evaluated in the same way transformations are. So, when we call sc.textFile(), the data is not loaded until it is necessary. As with transformations, the operation (in this case, reading the data) can
occur multiple times.</p>

<p>Although transformations are lazy, you can force Spark to execute them at any time by running an action, such as count(). This is an easy way to test out just part of your program.</p>

<p>Spark uses lazy evaluation to reduce the number of passes it has to take over our data by grouping operations together. In systems like Hadoop MapReduce, developers often have to spend a lot of time considering how to group together operations to minimize the number of MapReduce passes. In Spark, there is no substantial benefit to writing a single complex map instead of chaining together many simple operations. Thus, users are free to organize their program into smaller, more manageable operations.</p>

In [44]:
lazy_example = example_rdd.map(lambda x: x + 5)
#nothing actually happends here; Spark is waiting for an action so that it 
#can evaluate things in the most efficient way

In [45]:
lazy_example.count()
# things happen now

8